<a href="https://colab.research.google.com/github/Leixb/UPC-ADSDB_e2e/blob/master/notebooks/landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
import glob
import os
from datetime import datetime
import json

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  is_local = False
except ModuleNotFoundError:
  is_local = True

MessageError: ignored

In [ ]:
folder_landing = if is_local "./landing" else "/content/drive/MyDrive/ADSDB/landing"

folder_temporal = os.path.join(folder_landing, "temporal")
folder_persistent = os.path.join(folder_landing, "persistent")

extract_dir = os.path.join(folder_persistent, "extracted")

In [ ]:
for zip_file in glob.glob(os.path.join(folder_temporal, "*.zip")):
  print(zip_file)

  filename = os.path.basename(zip_file[:-4])
  timestamp = os.path.getmtime(zip_file)

  out_dir = os.path.join(extract_dir, f"{filename}-{timestamp}")
  os.makedirs(out_dir, exist_ok = True)
  print(out_dir)

  with ZipFile(zip_file, 'r') as zipObj:
    zipObj.extractall(out_dir)

  with open(os.path.join(out_dir, "metadata.json"), 'w') as outfile:
    json.dump({
        "filename": filename,
        "timestamp" : timestamp,
        "ingestion_time" : datetime.now().timestamp()
    }, outfile)

/content/drive/MyDrive/ADSDB/landing/temporal/morticd07.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/morticd07-1614616491.0
/content/drive/MyDrive/ADSDB/landing/temporal/morticd10_part3.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/morticd10_part3-1614616495.0
/content/drive/MyDrive/ADSDB/landing/temporal/morticd09.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/morticd09-1614616501.0
/content/drive/MyDrive/ADSDB/landing/temporal/morticd10_part2.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/morticd10_part2-1614616504.0
/content/drive/MyDrive/ADSDB/landing/temporal/morticd08.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/morticd08-1614684283.0
/content/drive/MyDrive/ADSDB/landing/temporal/mort_country_codes.zip
/content/drive/MyDrive/ADSDB/landing/persistent/extracted/mort_country_codes-1614687996.0
/content/drive/MyDrive/ADSDB/landing/temporal/morticd10_part1.zip
/content/drive/MyDrive/ADSDB/landing/persisten